1. Giải bài toán 8 ô chữ bằng A*


In [14]:
pip install simpleai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simpleai: filename=simpleai-0.8.3-py3-none-any.whl size=100982 sha256=df5363236887f8def2ad81e846f17126d8927aacb33f96fb4a9596f2a241cfd7
  Stored in directory: /root/.cache/pip/wheels/04/84/76/e493a82b60c25f283e4d315ac3b9b9ff44470556da775c58ad
Successfully built simpleai


In [17]:
import time
from simpleai.search import SearchProblem, astar  # Thư viện SimpleAI cho bài toán tìm kiếm và thuật toán A*
from simpleai.search.viewers import BaseViewer  # Để theo dõi thống kê tìm kiếm (số nút mở rộng, sinh ra)
from copy import deepcopy  # Để sao chép trạng thái khi cần

# ------------------------------------------------------------
# BÀI TOÁN 8-PUZZLE (Sử dụng SimpleAI)
# ------------------------------------------------------------

class EightPuzzleProblem(SearchProblem):
    def __init__(self, initial=None, goal=None, heuristic_func=None):
        """
        Khởi tạo bài toán 8-puzzle bằng cách kế thừa từ SearchProblem của SimpleAI.
        Trạng thái là tuple 3x3 bất biến để dễ so sánh.
        heuristic_func: Hàm heuristic được truyền vào để sử dụng (h1 hoặc h2).
        """
        if initial is None:
            # Trạng thái ban đầu mặc định: gần trạng thái đích để dễ kiểm tra
            initial = ((1, 2, 3),
                       (0, 8, 4),
                       (7, 6, 5))
        if goal is None:
            # Trạng thái đích mặc định: ô trống (0) ở vị trí (1,1)
            goal = ((1, 2, 3),
                    (8, 0, 4),
                    (7, 6, 5))
        # Gọi hàm khởi tạo của lớp cha với trạng thái ban đầu
        super().__init__(initial_state=initial)
        self.goal = goal  # Lưu trạng thái đích
        self.heuristic_func = heuristic_func  # Lưu hàm heuristic (h1 hoặc h2)

    def actions(self, state):
        """
        Trả về danh sách các hành động hợp lệ từ trạng thái hiện tại.
        Mỗi hành động là tọa độ (i,j) của ô lân cận có thể hoán đổi với ô trống (0).
        Hành động đại diện cho vị trí mà ô trống sẽ di chuyển tới (lên, xuống, trái, phải).
        """
        # Tìm vị trí ô trống (giá trị 0) trong bảng 3x3
        empty_i = empty_j = None
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    empty_i, empty_j = i, j
                    break
            if empty_i is not None:
                break

        # Tạo danh sách các hành động (tọa độ ô lân cận hợp lệ)
        moves = []
        # Kiểm tra 4 hướng di chuyển của ô trống
        if empty_i > 0:  # Có thể di chuyển lên (ô trống không ở hàng đầu)
            moves.append((empty_i - 1, empty_j))  # Thêm tọa độ ô phía trên
        if empty_i < 2:  # Có thể di chuyển xuống (ô trống không ở hàng cuối)
            moves.append((empty_i + 1, empty_j))  # Thêm tọa độ ô phía dưới
        if empty_j > 0:  # Có thể di chuyển trái (ô trống không ở cột đầu)
            moves.append((empty_i, empty_j - 1))  # Thêm tọa độ ô bên trái
        if empty_j < 2:  # Có thể di chuyển phải (ô trống không ở cột cuối)
            moves.append((empty_i, empty_j + 1))  # Thêm tọa độ ô bên phải
        return moves

    def result(self, state, action):
        """
        Trả về trạng thái mới sau khi thực hiện hành động (hoán đổi ô trống với ô tại action).
        action: tọa độ (i,j) của ô cần đổi chỗ với ô trống.
        """
        # Chuyển trạng thái từ tuple sang list để dễ chỉnh sửa
        grid = [list(row) for row in state]

        # Tìm vị trí ô trống (0)
        empty_i = empty_j = None
        for i in range(3):
            for j in range(3):
                if grid[i][j] == 0:
                    empty_i, empty_j = i, j
                    break
            if empty_i is not None:
                break

        # Lấy tọa độ ô cần hoán đổi từ hành động
        tile_i, tile_j = action
        # Hoán đổi giá trị giữa ô trống và ô tại tọa độ action
        grid[empty_i][empty_j], grid[tile_i][tile_j] = grid[tile_i][tile_j], grid[empty_i][empty_j]

        # Chuyển lại từ list sang tuple bất biến để trả về
        return tuple(tuple(row) for row in grid)

    def cost(self, state1, action, state2):
        # Chi phí mỗi bước di chuyển là 1 (mỗi lần hoán đổi ô trống)
        return 1

    def is_goal(self, state):
        # Kiểm tra xem trạng thái hiện tại có phải là trạng thái đích không
        return state == self.goal

    def heuristic(self, state):
        """
        Hàm heuristic: Gọi hàm heuristic_func được truyền vào khi khởi tạo (h1 hoặc h2).
        """
        if self.heuristic_func:
            return self.heuristic_func(state, self.goal)
        return 0  # Mặc định trả về 0 nếu không có heuristic

# ------------------------------------------------------------
# HEURISTICS
# ------------------------------------------------------------

def h_misplaced(state, goal):
    """
    Heuristic h1: Đếm số ô (không tính ô trống) ở sai vị trí so với trạng thái đích.
    """
    count = 0
    for i in range(3):
        for j in range(3):
            val = state[i][j]
            # Nếu ô không phải ô trống (0) và không đúng vị trí so với mục tiêu
            if val != 0 and val != goal[i][j]:
                count += 1
    return count


def h_manhattan(state, goal):
    """
    Heuristic h2: Tính tổng khoảng cách Manhattan của mỗi ô tới vị trí đích của nó.
    Khoảng cách Manhattan = |Δx| + |Δy| giữa vị trí hiện tại và vị trí đích của ô.
    """
    # Tạo bản đồ ánh xạ giá trị ô (tile) tới vị trí đích (i,j) trong trạng thái goal
    goal_pos = {}
    for i in range(3):
        for j in range(3):
            val = goal[i][j]
            goal_pos[val] = (i, j)

    # Tính tổng khoảng cách Manhattan
    dist_sum = 0
    for i in range(3):
        for j in range(3):
            val = state[i][j]
            if val != 0:  # Bỏ qua ô trống
                # Lấy vị trí đích của ô có giá trị val
                gi, gj = goal_pos[val]
                # Cộng khoảng cách Manhattan: |i - gi| + |j - gj|
                dist_sum += abs(i - gi) + abs(j - gj)
    return dist_sum

# ------------------------------------------------------------
# HỖ TRỢ IN KẾT QUẢ
# ------------------------------------------------------------

def print_state(state):
    # Hàm hỗ trợ in trạng thái bảng 3x3
    for row in state:
        print(" ".join(str(x) for x in row))  # In mỗi hàng với các số cách nhau bằng khoảng trắng
    print()  # In dòng trống để phân tách

def print_stats(name, result, time_taken, stats):
    """
    Hàm in thống kê và đường đi của lời giải.
    stats từ BaseViewer cung cấp số nút mở rộng và sinh ra.
    """
    print(f"--- Kết quả với {name} ---")
    if result is None or result.state is None:
        print("Không tìm thấy lời giải.")
        return
    # Xây dựng đường đi từ node đích
    states = []
    actions = []
    node = result
    while node is not None:
        states.append(node.state)
        if node.action is not None:
            actions.append(node.action)
        node = node.parent
    states.reverse()
    actions.reverse()

    # Lấy thông tin thống kê từ BaseViewer
    nodes_expanded = stats.get('nodes_expanded', 0)
    nodes_generated = stats.get('nodes_created', 0)

    # In thông tin về lời giải và hiệu suất
    print(f"Chi phí lời giải (g): {result.cost}")
    print(f"Độ sâu lời giải: {result.depth}")
    print(f"Số nút mở rộng: {nodes_expanded}")
    print(f"Số nút sinh ra: {nodes_generated}")
    print(f"Thời gian: {time_taken:.6f} s")
    print("Đường đi (các trạng thái):")
    for s in states:
        for row in s:
            print(row)
        print("-")
    print("==========================")

def run_and_report():
    """
    Hàm chạy thử nghiệm và in kết quả cho cả hai heuristic.
    So sánh hiệu suất của h1 và h2 dựa trên thời gian và số nút mở rộng/sinh ra.
    """
    # Khởi tạo bài toán
    initial_state = ((1, 2, 3), (0, 8, 4), (7, 6, 5))
    goal_state = ((1, 2, 3), (8, 0, 4), (7, 6, 5))

    print("=== BÀI TOÁN 8-PUZZLE ===")
    print("Trạng thái đầu:")
    print_state(initial_state)
    print("Trạng thái đích:")
    print_state(goal_state)

    # Chạy A* với heuristic h1 (số ô sai vị trí)
    print("Chạy A* với h1 (Số quân sai vị trí)...")
    viewer_h1 = BaseViewer()  # Tạo viewer để theo dõi thống kê
    problem_h1 = EightPuzzleProblem(initial=initial_state, goal=goal_state, heuristic_func=h_misplaced)
    start_time = time.time()
    result_h1 = astar(problem_h1, viewer=viewer_h1)  # Truyền viewer vào astar
    time_h1 = time.time() - start_time
    print_stats("h1 (misplaced tiles)", result_h1, time_h1, viewer_h1.stats)

    # Chạy A* với heuristic h2 (khoảng cách Manhattan)
    print("Chạy A* với h2 (Manhattan distance)...")
    viewer_h2 = BaseViewer()  # Tạo viewer mới cho h2
    problem_h2 = EightPuzzleProblem(initial=initial_state, goal=goal_state, heuristic_func=h_manhattan)
    start_time = time.time()
    result_h2 = astar(problem_h2, viewer=viewer_h2)  # Truyền viewer vào astar
    time_h2 = time.time() - start_time
    print_stats("h2 (Manhattan)", result_h2, time_h2, viewer_h2.stats)

    # So sánh kết quả của hai heuristic
    if result_h1 is not None and result_h2 is not None:
        print("=== SO SÁNH ===")
        nodes_expanded_h1 = viewer_h1.stats.get('nodes_expanded', 0)
        nodes_generated_h1 = viewer_h1.stats.get('nodes_created', 0)
        nodes_expanded_h2 = viewer_h2.stats.get('nodes_expanded', 0)
        nodes_generated_h2 = viewer_h2.stats.get('nodes_created', 0)

        print(f"Chi phí lời giải (g) h1: {result_h1.cost} | h2: {result_h2.cost}")
        print(f"Độ sâu lời giải h1: {result_h1.depth} | h2: {result_h2.depth}")
        print(f"Số nút mở rộng h1: {nodes_expanded_h1} | h2: {nodes_expanded_h2}")
        print(f"Số nút sinh ra h1: {nodes_generated_h1} | h2: {nodes_generated_h2}")
        print(f"Thời gian (s) h1: {time_h1:.6f} | h2: {time_h2:.6f}")
        print("=> Thường thấy h2 mở rộng ít hơn và/hoặc nhanh hơn do thông tin hơn.")
    else:
        print("Không có lời giải ở một trong hai heuristic (bất thường nếu trạng thái solvable).")


if __name__ == "__main__":
    run_and_report()  # Chạy chương trình

=== BÀI TOÁN 8-PUZZLE ===
Trạng thái đầu:
1 2 3
0 8 4
7 6 5

Trạng thái đích:
1 2 3
8 0 4
7 6 5

Chạy A* với h1 (Số quân sai vị trí)...
--- Kết quả với h1 (misplaced tiles) ---
Chi phí lời giải (g): 1
Độ sâu lời giải: 1
Số nút mở rộng: 0
Số nút sinh ra: 0
Thời gian: 0.000202 s
Đường đi (các trạng thái):
(1, 2, 3)
(0, 8, 4)
(7, 6, 5)
-
(1, 2, 3)
(8, 0, 4)
(7, 6, 5)
-
Chạy A* với h2 (Manhattan distance)...
--- Kết quả với h2 (Manhattan) ---
Chi phí lời giải (g): 1
Độ sâu lời giải: 1
Số nút mở rộng: 0
Số nút sinh ra: 0
Thời gian: 0.000186 s
Đường đi (các trạng thái):
(1, 2, 3)
(0, 8, 4)
(7, 6, 5)
-
(1, 2, 3)
(8, 0, 4)
(7, 6, 5)
-
=== SO SÁNH ===
Chi phí lời giải (g) h1: 1 | h2: 1
Độ sâu lời giải h1: 1 | h2: 1
Số nút mở rộng h1: 0 | h2: 0
Số nút sinh ra h1: 0 | h2: 0
Thời gian (s) h1: 0.000202 | h2: 0.000186
=> Thường thấy h2 mở rộng ít hơn và/hoặc nhanh hơn do thông tin hơn.


2. Giải bài toán xếp Hậu 8x8 bằng GA

In [18]:
from simpleai.search import SearchProblem, genetic
import random
import math

# ------------------------------------------------------------
# ĐỊNH NGHĨA LỚP BÀI TOÁN XẾP HẬU (N-Queens)
# ------------------------------------------------------------

class NQueensProblem(SearchProblem):
    def __init__(self, N=8):
        # Khởi tạo bài toán N-Queens với kích thước bảng N x N (mặc định N=8)
        # Trạng thái ban đầu là một tuple ngẫu nhiên, mỗi phần tử đại diện cho hàng của quân hậu trong một cột
        # Ví dụ: (2, 4, 1, ...) nghĩa là quân hậu ở cột 0 nằm ở hàng 2, cột 1 ở hàng 4, v.v.
        super().__init__(initial_state=tuple(random.randint(0, N-1) for _ in range(N)))
        self.N = N  # Lưu kích thước bảng

    def actions(self, state):
        """
        Trả về danh sách các hành động hợp lệ từ trạng thái hiện tại.
        Không cần thiết cho thuật toán di truyền (Genetic Algorithm - GA), nhưng yêu cầu bởi lớp SearchProblem.
        Vì GA không sử dụng actions, trả về danh sách rỗng.
        """
        return []

    def result(self, state, action):
        """
        Trả về trạng thái mới sau khi thực hiện hành động.
        Không cần thiết cho GA, nên trả về trạng thái hiện tại mà không thay đổi.
        """
        return state

    def value(self, state):
        """
        Hàm đánh giá (fitness function): tính số cặp quân hậu không tấn công nhau.
        - Mỗi trạng thái là một tuple đại diện cho vị trí hàng của quân hậu trong mỗi cột.
        - Xung đột xảy ra khi hai quân hậu ở cùng hàng hoặc cùng đường chéo.
        - Fitness tối đa là C(N,2) = N*(N-1)/2 (số cặp quân hậu có thể có).
        - Trả về số cặp không xung đột (cao hơn = tốt hơn).
        """
        conflicts = 0  # Đếm số cặp quân hậu xung đột
        for i in range(self.N):
            for j in range(i + 1, self.N):
                # Kiểm tra xung đột:
                # - Cùng hàng: state[i] == state[j]
                # - Cùng đường chéo: |state[i] - state[j]| == |i - j|
                if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                    conflicts += 1
        # Fitness = số cặp không xung đột = tổng số cặp - số xung đột
        return (self.N * (self.N - 1) // 2) - conflicts

    def generate_random_state(self):
        """
        Tạo một trạng thái ngẫu nhiên cho thuật toán di truyền.
        Trả về một tuple có N phần tử, mỗi phần tử là số ngẫu nhiên từ 0 đến N-1,
        đại diện cho hàng của quân hậu trong mỗi cột.
        """
        return tuple(random.randint(0, self.N-1) for _ in range(N))

    def crossover(self, state1, state2):
        """
        Hàm lai ghép (crossover): tạo trạng thái con từ hai trạng thái cha mẹ.
        - Cắt ngẫu nhiên tại một điểm (cut_point).
        - Lấy nửa đầu từ state1 và nửa sau từ state2.
        - Trả về trạng thái con mới dưới dạng tuple.
        """
        cut_point = random.randint(1, self.N - 1)  # Chọn điểm cắt ngẫu nhiên
        new_state = state1[:cut_point] + state2[cut_point:]  # Kết hợp hai nửa
        return new_state

    def mutate(self, state):
        """
        Hàm đột biến (mutation): thay đổi ngẫu nhiên một phần tử trong trạng thái.
        - Chọn ngẫu nhiên một cột (col).
        - Thay đổi hàng của quân hậu trong cột đó thành một giá trị ngẫu nhiên từ 0 đến N-1.
        - Trả về trạng thái mới dưới dạng tuple.
        """
        col = random.randint(0, self.N - 1)  # Chọn cột ngẫu nhiên
        new_row = random.randint(0, self.N - 1)  # Chọn hàng ngẫu nhiên
        state_list = list(state)  # Chuyển tuple thành list để chỉnh sửa
        state_list[col] = new_row  # Thay đổi hàng của quân hậu tại cột col
        return tuple(state_list)  # Chuyển lại thành tuple


# ------------------------------------------------------------
# HÀM CHẠY THUẬT TOÁN DI TRUYỀN N LẦN
# ------------------------------------------------------------

def run_genetic_n_times(n_runs=10, population_size=100, mutation_chance=0.05, iterations_limit=1000):
    """
    Chạy thuật toán di truyền (GA) n lần và trả về kết quả tốt nhất.
    - n_runs: số lần chạy thuật toán.
    - population_size: kích thước quần thể (số trạng thái trong mỗi thế hệ).
    - mutation_chance: xác suất đột biến.
    - iterations_limit: số lần lặp tối đa cho mỗi lần chạy GA.
    - Trả về: trạng thái tốt nhất, giá trị fitness tốt nhất, và lần chạy tốt nhất.
    """
    best_state = None  # Trạng thái tốt nhất
    best_value = -1    # Giá trị fitness tốt nhất
    best_run = -1      # Lần chạy cho ra kết quả tốt nhất
    max_fitness = 28   # Với N=8, C(8,2) = 28 (số cặp quân hậu không xung đột tối đa)

    # Chạy GA n lần
    for run in range(n_runs):
        # Tạo bài toán mới với trạng thái ban đầu ngẫu nhiên
        problem = NQueensProblem()
        # Chạy thuật toán di truyền với các tham số được chỉ định
        result = genetic(
            problem,
            population_size=population_size,
            mutation_chance=mutation_chance,
            iterations_limit=iterations_limit
        )
        # Tính giá trị fitness của trạng thái kết quả
        result_value = problem.value(result.state)

        # In kết quả của lần chạy hiện tại
        print(f"Run {run + 1}: State = {result.state}, Fitness = {result_value}/{max_fitness}")

        # Cập nhật kết quả tốt nhất nếu fitness cao hơn
        if result_value > best_value:
            best_value = result_value
            best_state = result.state
            best_run = run + 1

    return best

3. Bài toán xếp Hậu 8x8 bằng SA-hillclimbing


In [19]:
from simpleai.search import SearchProblem, simulated_annealing
import random
import math

# Định nghĩa lớp bài toán xếp hậu
class NQueensProblem(SearchProblem):
    def __init__(self):
        # Khởi tạo trạng thái ban đầu ngẫu nhiên: mỗi cột có một quân hậu ở hàng ngẫu nhiên
        super().__init__(initial_state=tuple(random.randint(0, 7) for _ in range(8)))
        self.N = 8

    def actions(self, state):
        # Các hành động: thay đổi vị trí hàng của một quân hậu trong một cột
        actions = []
        for col in range(self.N):
            for row in range(self.N):
                if row != state[col]:
                    new_state = list(state)
                    new_state[col] = row
                    actions.append(tuple(new_state))
        return actions

    def result(self, state, action):
        # Trả về trạng thái mới sau khi áp dụng hành động
        return action

    def value(self, state):
        # Tính giá trị của trạng thái: số cặp quân hậu không tấn công nhau
        conflicts = 0
        for i in range(self.N):
            for j in range(i + 1, self.N):
                # Kiểm tra nếu hai quân hậu tấn công nhau (cùng hàng hoặc cùng đường chéo)
                if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                    conflicts += 1
        # Giá trị là số cặp không xung đột: 28 - số xung đột
        return 28 - conflicts

    def generate_random_state(self):
        # Tạo trạng thái ngẫu nhiên
        return tuple(random.randint(0, 7) for _ in range(8))

# Hàm chạy Simulated Annealing nhiều lần và trả về kết quả tốt nhất
def run_simulated_annealing_n_times(n_runs=10):
    best_state = None
    best_value = -1
    best_run = -1

    for run in range(n_runs):
        # Tạo bài toán mới với trạng thái ngẫu nhiên
        problem = NQueensProblem()
        # Chạy thuật toán Simulated Annealing
        result = simulated_annealing(problem, iterations_limit=1000)
        result_value = problem.value(result.state)

        print(f"Run {run + 1}: State = {result.state}, Value = {result_value}")

        # Cập nhật kết quả tốt nhất
        if result_value > best_value:
            best_value = result_value
            best_state = result.state
            best_run = run + 1

    return best_state, best_value, best_run

# Chạy thuật toán 10 lần và in kết quả tốt nhất
best_state, best_value, best_run = run_simulated_annealing_n_times(10)
print("\nKết quả tốt nhất:")
print(f"Run {best_run}: State = {best_state}, Value = {best_value}")
if best_value == 28:
    print("Đây là nghiệm tối ưu: Không có quân hậu nào tấn công nhau!")
else:
    print(f"Còn {28 - best_value} cặp quân hậu tấn công nhau.")

Run 1: State = (2, 5, 1, 4, 7, 0, 3, 6), Value = 27
Run 2: State = (3, 1, 4, 2, 0, 2, 7, 5), Value = 27
Run 3: State = (4, 1, 1, 5, 2, 6, 3, 0), Value = 27
Run 4: State = (6, 2, 7, 1, 4, 0, 5, 3), Value = 28
Run 5: State = (7, 3, 0, 2, 5, 1, 6, 4), Value = 28
Run 6: State = (5, 2, 6, 1, 3, 7, 0, 4), Value = 28
Run 7: State = (5, 5, 1, 6, 0, 2, 4, 7), Value = 27
Run 8: State = (6, 4, 1, 1, 5, 7, 2, 0), Value = 27
Run 9: State = (2, 4, 1, 7, 0, 6, 3, 5), Value = 28
Run 10: State = (1, 4, 0, 5, 7, 2, 6, 3), Value = 27

Kết quả tốt nhất:
Run 4: State = (6, 2, 7, 1, 4, 0, 5, 3), Value = 28
Đây là nghiệm tối ưu: Không có quân hậu nào tấn công nhau!


4. Bài toán caro 3x3

In [20]:
!pip install easyAI

In [21]:
from easyAI import TwoPlayerGame, AI_Player, Negamax
from easyAI.Player import Human_Player

class GameController(TwoPlayerGame):
    def __init__(self, players):
        # Define the players
        self.players = players
        # Define who starts the game
        self.current_player = 1  # Use current_player instead of nplayer for clarity
        # Define the board (0 = empty, 1 = player 1, 2 = player 2)
        self.board = [0] * 9

    # Define possible moves
    def possible_moves(self):
        return [a + 1 for a, b in enumerate(self.board) if b == 0]

    # Make a move
    def make_move(self, move):
        self.board[int(move) - 1] = self.current_player

    # Check if the opponent has three in a line (loss condition for current player)
    def loss_condition(self):
        possible_combinations = [
            [1, 2, 3], [4, 5, 6], [7, 8, 9],  # Rows
            [1, 4, 7], [2, 5, 8], [3, 6, 9],  # Columns
            [1, 5, 9], [3, 5, 7]              # Diagonals
        ]
        return any([all([(self.board[i-1] == self.opponent_index)
                        for i in combination])
                   for combination in possible_combinations])

    # Check if the game is over
    def is_over(self):
        return (self.possible_moves() == []) or self.loss_condition()

    # Show current board position
    def show(self):
        print('\n' + '\n'.join([' '.join([['.', 'O', 'X'][self.board[3*j + i]]
                                          for i in range(3)]) for j in range(3)]))

    # Compute the score
    def scoring(self):
        return -100 if self.loss_condition() else 0

if __name__ == "__main__":
    # Define the algorithm (Negamax with depth 7)
    algorithm = Negamax(7)
    # Start the game with a human player and an AI player
    game = GameController([Human_Player(), AI_Player(algorithm)])
    game.play()


. . .
. . .
. . .

Player 1 what do you play ? 1

Move #1: player 1 plays 1 :

O . .
. . .
. . .

Move #2: player 2 plays 5 :

O . .
. X .
. . .

Player 1 what do you play ? 2

Move #3: player 1 plays 2 :

O O .
. X .
. . .

Move #4: player 2 plays 3 :

O O X
. X .
. . .

Player 1 what do you play ? 7

Move #5: player 1 plays 7 :

O O X
. X .
O . .

Move #6: player 2 plays 4 :

O O X
X X .
O . .

Player 1 what do you play ? 5

Player 1 what do you play ? 6

Move #7: player 1 plays 6 :

O O X
X X O
O . .

Move #8: player 2 plays 8 :

O O X
X X O
O X .

Player 1 what do you play ? 9

Move #9: player 1 plays 9 :

O O X
X X O
O X O
